# Google Colab Notebook for YOLOv11 Emotion Detection Model (GPU Test)

## Step 1: Set up the environment

In [1]:
!nvidia-smi  # Verify GPU availability

Wed Jul 16 17:55:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics # Install required packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [ ]:
import os
import shutil
from ultralytics import YOLO
import yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

Mounted at /content/drive


## Step 2: Upload and unzip dataset from device

In [5]:
# Check if ZIP file exists and unzip
zip_path = "/content/test-4.zip"
if not os.path.exists(zip_path):
    raise FileNotFoundError(f"Dataset ZIP file not found at {zip_path}. Please upload it to Colab.")
!unzip -q {zip_path} -d /content

## Step 3: Verify and set dataset path

In [6]:
# Device uploads to /content/test-4
dataset_path = "/content/test-4"
test_img_dir = os.path.join(dataset_path, "train", "images")
test_label_dir = os.path.join(dataset_path, "train", "labels")

In [7]:
# Check if test directories exist
if not os.path.exists(test_img_dir) or not os.path.exists(test_label_dir):
    raise FileNotFoundError(f"Test directories not found at {test_img_dir} or {test_label_dir}")

## Step 4: Update data.yaml with absolute paths

In [8]:
data_yaml_path = os.path.join(dataset_path, "data.yaml")
with open(data_yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

# Use absolute paths for the test set (use train folder as the test set)
data_yaml["train"] = os.path.abspath(os.path.join(dataset_path, "train/images"))
data_yaml["val"] = ""  # No validation set
data_yaml["test"] = os.path.abspath(os.path.join(dataset_path, "train/images"))  # Use all images for testing
data_yaml["nc"] = 7
data_yaml["names"] = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f)

In [9]:
# Verify paths exist
if not os.path.exists(data_yaml["test"]):
    raise FileNotFoundError(f"Test image directory not found: {data_yaml['test']}")

## Step 5: Load fine-tuned YOLOv11 model

In [10]:
model = YOLO("/content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_Emotion_Detection.pt")

## Step 6: Batch evaluation and inference

In [12]:
# Perform evaluation (mAP) and inference on the test set
val_results = model.val(
    data=data_yaml_path,
    batch=32,  # Batch size for T4 GPU
    conf=0.25,  # Confidence threshold for inference
    save=True,  # Save inference results
    split="test"  # Use test set (all 536 images)
)

# Extract and print mAP metrics
print("Test Metrics:")
print(f"mAP@50: {val_results.box.map50:.4f}")
print(f"mAP@50:95: {val_results.box.map:.4f}")

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2531.5±361.8 MB/s, size: 229.6 KB)


val: Scanning /content/test-4/train/labels.cache... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:17<00:00,  1.00s/it]


                   all        536      19056      0.206      0.215       0.16      0.111
                 Angry        523       1859      0.201      0.114      0.121     0.0819
               Disgust        354        701      0.182      0.173      0.117     0.0892
                  Fear        188        292    0.00931     0.0274    0.00498    0.00146
                 Happy        536       2342      0.228      0.418      0.251      0.177
               Neutral        536      10305      0.546      0.607      0.462      0.305
                   Sad        514       2310      0.158     0.0281     0.0815      0.064
              Surprise        494       1247      0.117      0.134     0.0853     0.0565
Speed: 1.1ms preprocess, 5.4ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs/detect/val2
Test Metrics:
mAP@50: 0.1603
mAP@50:95: 0.1106


In [13]:
# Extract speed metrics
preprocess_time = val_results.speed['preprocess']  # ms per image
inference_time = val_results.speed['inference']    # ms per image
postprocess_time = val_results.speed['postprocess']  # ms per image
total_time_per_image = preprocess_time + inference_time + postprocess_time  # ms

# Calculate FPS (per image)
fps = 1000 / total_time_per_image if total_time_per_image > 0 else 0

# Calculate inference time per face, latency per face, and FPS per face
num_images = 536  # Total images in dataset
num_instances = sum(val_results.nt_per_class)  # Total instances (e.g., 3859 previously)
avg_faces_per_image = num_instances / num_images if num_images > 0 else 0
inference_time_per_face = inference_time / avg_faces_per_image if avg_faces_per_image > 0 else 0
latency_per_face = total_time_per_image / avg_faces_per_image if avg_faces_per_image > 0 else 0
fps_per_face = 1000 / inference_time_per_face if inference_time_per_face > 0 else 0

# Print speed metrics, FPS, and per-face metrics
print("\nSpeed Metrics (per image):")
print(f"Preprocess: {preprocess_time:.1f}ms")
print(f"Inference: {inference_time:.1f}ms")
print(f"Postprocess: {postprocess_time:.1f}ms")
print(f"Total (Latency): {total_time_per_image:.1f}ms")
print(f"FPS: {fps:.2f}")
print(f"\nPer Face Metrics:")
print(f"Average faces per image: {avg_faces_per_image:.2f}")
print(f"Inference time per face: {inference_time_per_face:.3f}ms")
print(f"Latency per face: {latency_per_face:.3f}ms")
print(f"FPS per face: {fps_per_face:.2f}")


Speed Metrics (per image):
Preprocess: 1.1ms
Inference: 5.4ms
Postprocess: 3.8ms
Total (Latency): 10.3ms
FPS: 96.83

Per Face Metrics:
Average faces per image: 35.55
Inference time per face: 0.153ms
Latency per face: 0.290ms
FPS per face: 6539.11


In [14]:
drive_save_path = "/content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_RT-FER_Test_GPU"
os.makedirs(drive_save_path, exist_ok=True)  # Ensure directory exists

In [15]:
# Save metrics to Google Drive
metrics_file = os.path.join(drive_save_path, "test_metrics.txt")
os.makedirs(os.path.dirname(metrics_file), exist_ok=True)  # Ensure parent directory exists
with open(metrics_file, "w") as f:
    f.write(f"Test Metrics:\n")
    f.write(f"mAP@50: {val_results.box.map50:.4f}\n")
    f.write(f"mAP@50:95: {val_results.box.map:.4f}\n")
    f.write(f"\nSpeed Metrics (per image):\n")
    f.write(f"Preprocess: {preprocess_time:.1f}ms\n")
    f.write(f"Inference: {inference_time:.1f}ms\n")
    f.write(f"Postprocess: {postprocess_time:.1f}ms\n")
    f.write(f"Total (Latency): {total_time_per_image:.1f}ms\n")
    f.write(f"FPS: {fps:.2f}\n")
    f.write(f"\nPer Face Metrics:\n")
    f.write(f"Average faces per image: {avg_faces_per_image:.2f}\n")
    f.write(f"Inference time per face: {inference_time_per_face:.3f}ms\n")
    f.write(f"Latency per face: {latency_per_face:.3f}ms\n")
    f.write(f"FPS per face: {fps_per_face:.2f}\n")
print(f"Metrics saved to {metrics_file}")

Metrics saved to /content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_RT-FER_Test_GPU/test_metrics.txt


## Step 7: Save inference results to Google Drive

In [17]:
inference_results_dir = "runs/detect/val"
drive_inference_save_path = os.path.join(drive_save_path, "inference_results")
os.makedirs(drive_inference_save_path, exist_ok=True)  # Ensure inference directory exists
shutil.copytree(inference_results_dir, drive_inference_save_path, dirs_exist_ok=True)
print(f"Inference results saved to {drive_inference_save_path}")

Inference results saved to /content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_RT-FER_Test_GPU/inference_results
